In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile
import os
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip', 'r') as train_z:
    train_z.extractall('/kaggle/working/')
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/test1.zip', 'r') as test_z:
    test_z.extractall('/kaggle/working/')

In [ ]:
print(os.listdir('/kaggle/working/'))

In [ ]:
#print(os.listdir('/kaggle/working/train'))

filenames = os.listdir('/kaggle/working/train')
categories = [filename.split('.')[0] for filename in filenames]

train_df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})
train_df['label'] = np.where(train_df['category'] == 'cat', 1, 0)
train_df.head()

In [ ]:
filenames = os.listdir('/kaggle/working/test1')
categories = [filename.split('.')[0] for filename in filenames]

test_df = pd.DataFrame({
    'filename': filenames,
})
test_df.head()

In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42) 
print('Train set shape: {}  Test set shape: {}'.format(train_df.shape, val_df.shape))

In [ ]:
import tensorflow as tf

IMAGE_SIZE=(128, 128)
train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=15, rescale=1./255, shear_range=0.1, zoom_range=0.2, horizontal_flip=True, width_shift_range=0.1, height_shift_range=0.1)
train_generator = train_data_gen.flow_from_dataframe(train_df, '/kaggle/working/train/', x_col='filename', y_col='category', class_mode='categorical', target_size=IMAGE_SIZE, batch_size=32)

val_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_generator = val_data_gen.flow_from_dataframe(val_df, '/kaggle/working/train/', x_col='filename', y_col='category', class_mode='categorical', target_size=IMAGE_SIZE, batch_size=32)


In [ ]:
network = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    
    tf.keras.layers.Conv2D(64, (3,3), activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    
    tf.keras.layers.Conv2D(128, (3,3), activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(2, activation='softmax')
])
network.build(input_shape=(None, 128, 128, 3))
network.summary()

In [ ]:
TRAIN_NUM_EXAMPLES = 20000
VAL_NUM_EXAMPLES = 5000

In [ ]:
network.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
history = network.fit_generator(generator=train_generator, validation_data=val_generator, epochs=10, callbacks=[learning_rate_reduction], steps_per_epoch=TRAIN_NUM_EXAMPLES//32, validation_steps=VAL_NUM_EXAMPLES//32)
history.history

In [ ]:
test_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = train_data_gen.flow_from_dataframe(test_df, '/kaggle/working/test1/', x_col='filename', y_col=None, class_mode=None, target_size=IMAGE_SIZE, batch_size=32)
predictions = network.predict_generator(test_generator, steps=test_df.shape[0]//10)
test_df['category'] = np.argmax(predictions, axis=-1)
test_df.head()

In [ ]:
submission_df = test_df.copy()
submission_df['id'] = submission_df['filename'].str.split('.').str[0]
submission_df['label'] = submission_df['category']
submission_df.drop(['filename', 'category'], axis=1, inplace=True)
submission_df.to_csv('submission.csv', index=False)

In [ ]:
print(os.listdir('/kaggle/working/'))